In [3]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_31.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "31_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "31_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "31_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "31_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "31_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "31_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 31_sub_mat_1.csv has been saved.
Submatrix 31_sub_mat_2.csv has been saved.
Submatrix 31_sub_mat_3.csv has been saved.
submatrix 31_cross_matrix_1_2.csv has been saved
submatrix 31_cross_matrix_1_3.csv has been saved
submatrix 31_cross_matrix_2_3.csv has been saved


38099.0   38101.0   38103.0   38105.0   39001.0   39003.0   39005.0  \
21083.0  0.664733  -0.72072 -0.683733 -0.682238 -0.564953 -0.651889 -0.644007   
21085.0  0.756125  -0.76929  -0.83144 -0.725862 -0.661414 -0.684955 -0.783482   
21087.0  0.827044  0.750839 -0.874292 -0.745337 -0.676704 -0.672622 -0.805147   
21089.0  0.782265 -0.678808 -0.791337 -0.746587 -0.668611 -0.661542 -0.778545   
21091.0  0.664588 -0.684797  0.660904 -0.763224 -0.634729  -0.60778 -0.689641   
...           ...       ...       ...       ...       ...       ...       ...   
38089.0 -0.792713  1.002753 -0.831338  0.856474 -0.688048  0.824742 -0.723307   
38091.0  0.686902 -0.784278 -0.675001 -0.763406 -0.610289 -0.727049 -0.703554   
38093.0  0.964042  0.935502   0.94367  0.914463 -0.768066  0.828802 -0.882646   
38095.0  0.761661  0.761251  0.780659  0.833418 -0.749509 -0.766121 -0.807246   
38097.0  0.816215 -0.884288  0.811349 -0.879467 -0.722657 -0.851526 -0.800276   

          39007.0   39009.0   39011.0  ...   56027.0   56029.0   56031.0  \
21083.0 -0.625022 -0.494858 -0.656541  ...  0.394142 -0.676024  0.670454   
21085.0 -0.778803 -0.679754 -0.691526  ...  0.330206 -0.828002   0.82869   
21087.0 -0.777386 -0.694559 -0.706268  ... -0.249709  -0.85332  0.893221   
21089.0 -0.722211 -0.701087 -0.668792  ... -0.240803 -0.742509  0.756598   
21091.0 -0.646889 -0.620757 -0.624103  ... -0.382429  0.659357  0.603708   
...           ...       ...       ...  ...       ...       ...       ...   
38089.0 -0.824243 -0.561309   0.85253  ...  0.471488 -0.889064 -0.981555   
38091.0  -0.71121 -0.585443 -0.732142  ...  0.549768  -0.73206 -0.705568   
38093.0 -0.865825 -0.767276  -0.85231  ... -0.405663  0.934682  0.920661   
38095.0 -0.828954 -0.769333 -0.782557  ... -0.402824  -0.83103 -0.688344   
38097.0 -0.788167 -0.651188 -0.829796  ...  0.593653 -0.800116  0.740424   

          56033.0   56035.0   56037.0   56039.0   56041.0   56043.0   56045.0  
21083.0 -0.742427  0.635443 -0.701499 -0.716263 -0.734586 -0.508241 -0.558108  
21085.0  0.745483  0.780492 -0.649578 -0.814901 -0.849684 -0.449904 -0.560884  
21087.0  0.741885  0.857281 -0.595951  -0.85582 -0.790375 -0.472425 -0.624761  
21089.0  0.616783  0.769805 -0.604937 -0.776262  0.736141 -0.479475 -0.571992  
21091.0 -0.657037   0.64942 -0.641259  0.699398  0.715426 -0.474454  -0.63702  
...           ...       ...       ...       ...       ...       ...       ...  
38089.0   1.03762 -0.729425  0.900928  0.812473  0.860475   0.52565 -0.787261  
38091.0  0.715465  0.615224 -0.762277 -0.760428 -0.788981  0.657263  -0.75591  
38093.0  0.878557   0.83976 -0.824837  0.933422  0.949126  0.582951 -0.736832  
38095.0 -0.698091  0.726125 -0.752226   0.81925 -0.813444  -0.55997 -0.753988  
38097.0 -0.921913 -0.713821 -0.928656  0.859716 -0.863748 -0.749212 -0.765038  

[1000 rows x 1104 columns]